In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import supertrend_lib as stlib
import sbeta_lib as sbeta
import smacross_lib as smacross
import CAPM_lib as capm
import LR_run_model_lib as LR_run
import LR_model_lib as LR_models

In [2]:
def add_update(eval_df,values):
    """ Add a a row to dataframe from a list of data points in the same sequence as the columns

    Args:
        eval_df (_type_): DataFrame receiving the new row
        values_list (_type_): A list of values in the same data types and sequance order as the dataframe columns

    Returns:
        _type_: The updated dataframe
    """
    tmp_df = pd.DataFrame([values], columns=eval_df.columns)
    eval_df = pd.concat([eval_df, tmp_df], ignore_index=True)
    return eval_df

In [3]:
def read_stocklist(filename):
    with open(filename) as f:
        stocklist = f.read()
    return stocklist.strip().split(",")


In [4]:
def update_models(stock_list):
    ret = LR_models.update_list_of_models(stock_list=stock_list)
    models_df = pd.DataFrame({'Stock' : stock_list,
                              'Model' : [f"Model{i}" for i in ret]})
    
    # print(models_df)
    return models_df

In [5]:
# stocks_from_file = read_stocklist(filename='./stocks_list.txt')
# stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
# dfff = update_models(stocks_from_file)
# dfff

In [6]:
# dfff[dfff['Stock'] == 'AAPL']['Model'].values[0]

In [7]:
def run_supertrend(stock,start_date):
    return stlib.supertrend(stock,start_date=start_date)


In [8]:
def init_eval_table():
    columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 
               'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'        ,'SMA_FastXSlow'    , 'Beta'         , 
               'CAPM'              ,'Daily_VaR'  ]

    dtypes = [np.datetime64        , str                , float            , bool              , str               , str            , 
              str                  , str                , str              , np.datetime64     , str               , float          , 
              float                , float        ]

    # Initialize the DataFrame with empty rows
    eval_df = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        eval_df[col] = eval_df[col].astype(dtype)
        
    return eval_df

In [9]:
def first_date_N_years_ago(years):
    # Calculate start and end dates N years back
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date.replace(year=end_date.year - years, month=1, day=1)
    return start_date.strftime("%Y-%m-%d")


In [10]:
def load_sp500_list():
    stocks_list_csv = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_df = pd.DataFrame()
    
    try:
        wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
        data = wiki_data[0] # All data is stored in first cell
        sp_df = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
    except:
        print("Cannot open file", stocks_list_csv)

    # remove the dotted symbols, they are redundant 
    no_dot_symbols = [i for i in sp_df['Symbol'] if i.find('.')==-1]
    sp_df = sp_df[sp_df['Symbol'].isin(no_dot_symbols)]
    
    # returns count_row, count_col, df
    return sp_df.shape[0], sp_df.shape[1], sp_df


In [11]:
m,n,sp_df = load_sp500_list()
# sp_df[sp_df['Symbol'].isin([i for i in sp_df['Symbol'] if i.find('.')>0])]

In [12]:
def recommendation_table(eval_df,stock_list, lookback_years=1, sma_fast=40, sma_slow=200,run_update_models=False):
    stock_list = [str(s).upper() for s in stock_list]
    # print("Stock List:",stock_list)
    start_date = first_date_N_years_ago(lookback_years)
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    if run_update_models :
        stock_best_model_df = update_models(stock_list=stock_list)

    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data = run_supertrend(s,start_date)
        print(f"{s}",end=',')
        sma_sig, sma_date, fastXslow = smacross.sma_xing(stock_data,sma_fast,sma_slow)
        beta, market_data = sbeta.get_beta(stock_data)
        CAPM, VaR = capm.CAPM_VaR(stock_data=stock_data,market_data=market_data,bond_mat_duration = lookback_years,stock_beta=beta)
        LR_recommend = LR_run.get_recommendation(stock=s,lookback=lookback_years)
        LR_recommend_str = f"{LR_recommend[0]},{LR_recommend[1]},{LR_recommend[2]}"
        
        # columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 
                #     'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'        ,'SMA_FastXSlow'    , 'Beta'         , 
                #     'CAPM'              ,'Daily_VaR'  ]
        
        if run_update_models:
            model_str = stock_best_model_df[stock_best_model_df['Stock'] == s].Model.values[0]
        else:
            model_str = 'N/A'
            
        new_row = [today           , s                  , close_price      , winner            , buysell         ,    buysell_date ,    
                   model_str       , LR_recommend_str   , sma_sig          , sma_date          , fastXslow       ,    beta         , 
                   CAPM            , VaR  ]    
        
        eval_df = add_update(eval_df=eval_df,values = new_row)
        
    return eval_df

In [13]:
def eval_all_sp500(lookback_years = 1,sma_fast = 40, sma_slow = 200, regenerate_models = False ):
    n,m,sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_sp500, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [14]:
def eval_list_from_file(filename='./stocks_list.txt',lookback_years = 1,sma_fast = 40, sma_slow = 200 ,regenerate_models = False):
    stocks_from_file = read_stocklist(filename=filename)
    stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_from_file, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [15]:
def screen_for_buys(eval_df):
        buys_df = eval_df[ (eval_df['Supertrend_Winner']==True) &  
                (eval_df['Supertrend_Result']=='Buy') & 
                (eval_df['LR_Next_Day_Recomm'] == 'Buy,Buy,Buy') &
                (eval_df['SMA_Crossed_Up']=='Buy')].sort_values(by=['Supertrend_Winner','Supertrend_Result','ST_Signal_Date','SMA_Crossed_Up','SMA_X_Date'],
                                                                ascending=[False,True,False,True,False])
        
        return buys_df

### Evaluate Stocks List

In [16]:
regenerate_models = True
eval_df = eval_df_from_file = eval_list_from_file(filename='stocks_list.txt',lookback_years=2,regenerate_models=regenerate_models)
# eval_df_sp500 = eval_all_sp500(lookback_years=1)

### All Results

In [37]:
eval_df.sort_values(['ST_Signal_Date','SMA_X_Date'], ascending=False)

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
7,2023-03-26,MU,61.16,False,Buy,2023-03-23,Model2,"Sell,Buy,Buy",Buy,2023-02-27,"(40, 200)",1.5194,3.02,-6.45
0,2023-03-26,QQQ,310.89,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2714,3.14,-3.86
20,2023-03-26,ADBE,374.96,True,Buy,2023-03-17,Model2,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
14,2023-03-26,COIN,67.83,False,Buy,2023-03-17,Model2,"Sell,Sell,Sell",Buy,2023-03-09,"(40, 200)",2.7611,-13.22,-14.56
2,2023-03-26,MSFT,280.57,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2570,3.15,-4.43
11,2023-03-26,GOOGL,105.44,False,Buy,2023-03-16,Model2,"Sell,Sell,Sell",Sell,2022-03-02,"(40, 200)",1.3573,3.10,-4.99
8,2023-03-26,TSLA,190.41,True,Sell,2023-03-09,Model2,"Sell,Sell,Sell",Sell,2022-10-03,"(40, 200)",1.8310,2.87,-9.40
15,2023-03-26,MRVL,41.07,False,Sell,2023-03-07,Model2,"Buy,Buy,Buy",Sell,2022-04-08,"(40, 200)",2.1114,2.74,-8.19
9,2023-03-26,SPY,395.75,False,Sell,2023-03-01,Model2,"Buy,Buy,Buy",Buy,2023-02-07,"(40, 200)",1.0025,3.27,-2.88
10,2023-03-26,STX,61.64,False,Sell,2023-02-27,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.2095,3.17,-6.10


### Buy, Buy, and more Buys

In [20]:
buys_eval_df= screen_for_buys(eval_df=eval_df)
buys_eval_df

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
20,2023-03-26,ADBE,374.96,True,Buy,2023-03-17,Model2,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
19,2023-03-26,CDNS,204.48,True,Buy,2023-01-17,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.53
3,2023-03-26,NVDA,267.79,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1726,2.71,-8.48


### Save the Buys

In [21]:
eval_df.to_csv("./stocks_buys_eval_df_table.csv")

### All Roads Lead to UP & Safe

In [36]:
buys_safe = buys_eval_df[(buys_eval_df['Beta']>1) & (buys_eval_df['Beta']<1.5) ].sort_values('Daily_VaR',ascending=False)
buys_safe

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
19,2023-03-26,CDNS,204.48,True,Buy,2023-01-17,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.53
20,2023-03-26,ADBE,374.96,True,Buy,2023-03-17,Model2,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70


### Save the Safe-Buys as Top Picks

In [25]:
buys_safe.to_csv("./top_picks.csv")

### Will Go UP the Next Day

In [26]:
LR_Next_Day_Recomm_only = eval_df[eval_df['LR_Next_Day_Recomm']=='Buy,Buy,Buy'].sort_values('Daily_VaR',ascending=False)
LR_Next_Day_Recomm_only

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
9,2023-03-26,SPY,395.75,False,Sell,2023-03-01,Model2,"Buy,Buy,Buy",Buy,2023-02-07,"(40, 200)",1.0025,3.27,-2.88
0,2023-03-26,QQQ,310.89,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2714,3.14,-3.86
2,2023-03-26,MSFT,280.57,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2570,3.15,-4.43
18,2023-03-26,DE,386.50,True,Sell,2023-01-19,Model1,"Buy,Buy,Buy",Buy,2022-11-04,"(40, 200)",0.8237,3.36,-4.95
19,2023-03-26,CDNS,204.48,True,Buy,2023-01-17,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.53
20,2023-03-26,ADBE,374.96,True,Buy,2023-03-17,Model2,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
12,2023-03-26,AMZN,98.13,False,Sell,2023-02-09,Model2,"Buy,Buy,Buy",Sell,2022-01-19,"(40, 200)",1.5189,3.02,-5.84
15,2023-03-26,MRVL,41.07,False,Sell,2023-03-07,Model2,"Buy,Buy,Buy",Sell,2022-04-08,"(40, 200)",2.1114,2.74,-8.19
3,2023-03-26,NVDA,267.79,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1726,2.71,-8.48
16,2023-03-26,PLTR,8.20,False,Sell,2023-02-23,Model1,"Buy,Buy,Buy",Buy,2023-03-23,"(40, 200)",1.8091,2.88,-10.40


### AI Says: Next Trading Day they're Down

In [27]:
LR_Next_Day_Sell_only = eval_df[(eval_df['LR_Next_Day_Recomm']=='Sell,Buy,Buy') | 
                                (eval_df['LR_Next_Day_Recomm']=='Sell,Sell,Sell') | 
                                (eval_df['LR_Next_Day_Recomm']=='Sell,Buy,Sell') | 
                                (eval_df['LR_Next_Day_Recomm']=='Buy,Sell,Sell')]
LR_Next_Day_Sell_only

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
1,2023-03-26,AAPL,160.25,True,Buy,2023-01-23,Model2,"Buy,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2952,3.13,-4.55
4,2023-03-26,META,206.01,False,Buy,2022-11-10,Model1,"Sell,Sell,Sell",Buy,2023-02-23,"(40, 200)",1.6742,2.95,-7.94
5,2023-03-26,IBM,125.29,False,Sell,2022-12-15,Model1,"Sell,Buy,Sell",Sell,2023-03-09,"(40, 200)",0.5399,3.50,-3.48
6,2023-03-26,QCOM,124.76,False,Sell,2023-02-21,Model1,"Sell,Buy,Sell",Sell,2022-04-12,"(40, 200)",1.5281,3.02,-6.03
7,2023-03-26,MU,61.16,False,Buy,2023-03-23,Model2,"Sell,Buy,Buy",Buy,2023-02-27,"(40, 200)",1.5194,3.02,-6.45
8,2023-03-26,TSLA,190.41,True,Sell,2023-03-09,Model2,"Sell,Sell,Sell",Sell,2022-10-03,"(40, 200)",1.8310,2.87,-9.40
10,2023-03-26,STX,61.64,False,Sell,2023-02-27,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.2095,3.17,-6.10
11,2023-03-26,GOOGL,105.44,False,Buy,2023-03-16,Model2,"Sell,Sell,Sell",Sell,2022-03-02,"(40, 200)",1.3573,3.10,-4.99
13,2023-03-26,SQ,60.68,False,Sell,2023-02-10,Model2,"Sell,Sell,Sell",Buy,2023-02-07,"(40, 200)",2.5997,2.50,-10.66
14,2023-03-26,COIN,67.83,False,Buy,2023-03-17,Model2,"Sell,Sell,Sell",Buy,2023-03-09,"(40, 200)",2.7611,-13.22,-14.56


### On Up Supertrend

In [33]:
up_Supertrend = eval_df[eval_df['Supertrend_Result'] == 'Buy'].sort_values('ST_Signal_Date', ascending=False)
up_Supertrend

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
7,2023-03-26,MU,61.16,False,Buy,2023-03-23,Model2,"Sell,Buy,Buy",Buy,2023-02-27,"(40, 200)",1.5194,3.02,-6.45
0,2023-03-26,QQQ,310.89,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2714,3.14,-3.86
14,2023-03-26,COIN,67.83,False,Buy,2023-03-17,Model2,"Sell,Sell,Sell",Buy,2023-03-09,"(40, 200)",2.7611,-13.22,-14.56
20,2023-03-26,ADBE,374.96,True,Buy,2023-03-17,Model2,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
2,2023-03-26,MSFT,280.57,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2570,3.15,-4.43
11,2023-03-26,GOOGL,105.44,False,Buy,2023-03-16,Model2,"Sell,Sell,Sell",Sell,2022-03-02,"(40, 200)",1.3573,3.10,-4.99
1,2023-03-26,AAPL,160.25,True,Buy,2023-01-23,Model2,"Buy,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2952,3.13,-4.55
19,2023-03-26,CDNS,204.48,True,Buy,2023-01-17,Model2,"Buy,Buy,Buy",Buy,2022-12-19,"(40, 200)",1.3805,3.09,-5.53
3,2023-03-26,NVDA,267.79,True,Buy,2023-01-12,Model2,"Buy,Buy,Buy",Buy,2023-01-24,"(40, 200)",2.1726,2.71,-8.48
4,2023-03-26,META,206.01,False,Buy,2022-11-10,Model1,"Sell,Sell,Sell",Buy,2023-02-23,"(40, 200)",1.6742,2.95,-7.94


### Fast SMA Crossed Slow SMA

In [35]:
Crossed_up = eval_df[eval_df['SMA_Crossed_Up'] == 'Buy'].sort_values('SMA_X_Date', ascending=False)
Crossed_up

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
16,2023-03-26,PLTR,8.20,False,Sell,2023-02-23,Model1,"Buy,Buy,Buy",Buy,2023-03-23,"(40, 200)",1.8091,2.88,-10.40
20,2023-03-26,ADBE,374.96,True,Buy,2023-03-17,Model2,"Buy,Buy,Buy",Buy,2023-03-16,"(40, 200)",1.4486,3.06,-5.70
1,2023-03-26,AAPL,160.25,True,Buy,2023-01-23,Model2,"Buy,Sell,Sell",Buy,2023-03-13,"(40, 200)",1.2952,3.13,-4.55
2,2023-03-26,MSFT,280.57,False,Buy,2023-03-16,Model2,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2570,3.15,-4.43
14,2023-03-26,COIN,67.83,False,Buy,2023-03-17,Model2,"Sell,Sell,Sell",Buy,2023-03-09,"(40, 200)",2.7611,-13.22,-14.56
10,2023-03-26,STX,61.64,False,Sell,2023-02-27,Model2,"Sell,Sell,Sell",Buy,2023-03-08,"(40, 200)",1.2095,3.17,-6.10
0,2023-03-26,QQQ,310.89,False,Buy,2023-03-21,Model2,"Buy,Buy,Buy",Buy,2023-03-01,"(40, 200)",1.2714,3.14,-3.86
7,2023-03-26,MU,61.16,False,Buy,2023-03-23,Model2,"Sell,Buy,Buy",Buy,2023-02-27,"(40, 200)",1.5194,3.02,-6.45
4,2023-03-26,META,206.01,False,Buy,2022-11-10,Model1,"Sell,Sell,Sell",Buy,2023-02-23,"(40, 200)",1.6742,2.95,-7.94
9,2023-03-26,SPY,395.75,False,Sell,2023-03-01,Model2,"Buy,Buy,Buy",Buy,2023-02-07,"(40, 200)",1.0025,3.27,-2.88


### Test Single Stocks

In [28]:
single_stock_df = init_eval_table()
single_stock_df = recommendation_table(single_stock_df,['msft'], lookback_years=5, sma_fast=40, sma_slow=200)
single_stock_df


MSFT,

,Last_Run,Stock,Last_Price,Supertrend_Winner,Supertrend_Result,ST_Signal_Date,LR_Best_Model,LR_Next_Day_Recomm,SMA_Crossed_Up,SMA_X_Date,SMA_FastXSlow,Beta,CAPM,Daily_VaR
0,2023-03-26,MSFT,280.57,False,Buy,2023-03-16,N/A,"Buy,Buy,Buy",Buy,2023-03-13,"(40, 200)",1.2159,10.54,-4.73
